In [1]:
import os
import sys
import pandas as pd
import re

In [35]:
income = pd.read_csv("raw/MedianHouseholdIncome2015.csv", encoding= 'unicode_escape')
poverty = pd.read_csv("raw/PercentagePeopleBelowPovertyLevel.csv", encoding= 'unicode_escape')
highschool = pd.read_csv("raw/PercentOver25CompletedHighSchool.csv", encoding= 'unicode_escape')
race = pd.read_csv("raw/ShareRaceByCity.csv", encoding= 'unicode_escape')

death_cases = pd.read_csv("raw/PoliceKillingsUS.csv", encoding= 'unicode_escape')

In [36]:
df = income

df = pd.merge(df, poverty,  how='inner', on=['Geographic Area', 'City'])
df = pd.merge(df, highschool,  how='inner', on=['Geographic Area', 'City'])
df = pd.merge(df, race,  how='inner', left_on=['Geographic Area', 'City'],right_on =['Geographic area', 'City'] )
df = df.drop(columns = ['Geographic area'])
df.head()

,Geographic Area,City,Median Income,poverty_rate,percent_completed_hs,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,11207,78.8,21.2,67.2,30.2,0,0,1.6
1,AL,Abbeville city,25615,29.1,69.1,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,42575,25.5,78.9,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,37083,30.7,81.4,99.1,0.1,0,0.1,0.4
4,AL,Akron town,21667,42,68.6,13.2,86.5,0,0,0.3


In [37]:
df['City'] = [re.sub(' city','', str(x)) for x in df['City']]
df['City'] = [re.sub(' town','', str(x)) for x in df['City']]
df['City'] = [re.sub(' CDP','', str(x)) for x in df['City']]
df['City'] = [re.sub(' village','', str(x)) for x in df['City']]

In [38]:
# Join income, poverty, highschool, race to the target dataframe
df = pd.merge(death_cases, df,  how='inner', left_on=['state', 'city'], right_on = ['Geographic Area','City'])
df = df.drop(columns = ['Geographic Area','City'])

In [39]:
df

,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,flee,body_camera,Median Income,poverty_rate,percent_completed_hs,share_white,share_black,share_native_american,share_asian,share_hispanic
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,...,Not fleeing,False,37072,28.6,80.1,78.9,0.8,3.7,1.1,19.2
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,...,Not fleeing,False,65765,14.9,88.1,70.9,2.6,1,8.9,21.1
2,890,Phyllis Ilene Jepsen,02/10/15,shot,knife,55.0,F,W,Aloha,OR,...,Not fleeing,False,65765,14.9,88.1,70.9,2.6,1,8.9,21.1
3,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,...,Not fleeing,False,45947,17.3,87.5,71.9,11.5,1.2,4.8,15.3
4,765,Nicholas Garner,22/08/15,shot,vehicle,26.0,M,W,Wichita,KS,...,Car,False,45947,17.3,87.5,71.9,11.5,1.2,4.8,15.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,2809,Sergio Velasco-Martinez,27/07/17,shot,blunt object,25.0,M,NaN,Avenal,CA,...,Not fleeing,False,32432,36.4,48.8,39,10.5,1.2,0.7,71.8
2199,2811,Matthew Folden,27/07/17,shot,knife,31.0,M,NaN,Wenatchee,WA,...,Not fleeing,False,46865,14,81.4,76.7,0.4,1.2,1.1,29.4
2200,2812,Alejandro Alvarado,27/07/17,shot,knife,NaN,M,H,Chowchilla,CA,...,Not fleeing,False,34559,32.1,68.3,61.6,12.6,2,2.1,37.8
2201,2818,Dennis W. Robinson,29/07/17,shot,gun,48.0,M,NaN,Melba,ID,...,Car,False,41154,16.4,90.4,78.9,0.2,0,0.2,24.6


In [40]:
# Female = 0, Male = 1
df.loc[df['gender'] == 'F', 'gender'] = 0
df.loc[df['gender'] == 'M', 'gender'] = 1

# body_camera: FALSE = 0, TRUE = 1
df.loc[df['body_camera'] == False, 'body_camera'] = 0
df.loc[df['body_camera'] == True, 'body_camera'] = 1

# signs of mental illness: FALSE = 0, TRUE = 1
df.loc[df['signs_of_mental_illness'] == False, 'signs_of_mental_illness'] = 0
df.loc[df['signs_of_mental_illness'] == True, 'signs_of_mental_illness'] = 1

# flee: not fleeing = 0, otherwise 1
df.loc[df['flee'] != 'Not fleeing', 'flee'] = 1
df.loc[df['flee'] == 'Not fleeing', 'flee'] = 0

# armed: unarmed = 0, otherwise 1
df.loc[df['armed'] != 'unarmed', 'armed'] = 1
df.loc[df['armed'] == 'unarmed', 'armed'] = 0

# manner of death: shot = 0, shot and Tasered = 1
df.loc[df['manner_of_death'] == 'shot', 'manner_of_death'] = 0
df.loc[df['manner_of_death'] == 'shot and Tasered', 'manner_of_death'] = 1

In [41]:
# one hot encode the race column
one_hot = pd.get_dummies(df['race'])
df = df.drop('race',axis = 1)
df = df.join(one_hot)
df

# one hot encode the theat level column
one_hot = pd.get_dummies(df['threat_level'])
df = df.drop('threat_level',axis = 1)
df = df.join(one_hot)

In [42]:
df = df.rename(columns={'A': 'Race_Asian', 'B': 'Race_Black', 'H': 'Race_Hispanic', 
                        'N': 'Race_Native_American', 'O': 'Race_Other', 'W': 'Race_White'})
df = df.rename(columns={'attack': 'threat_attack', 'other': 'threat_other', 'undetermined': 'threat_undem'})

In [43]:
df

,id,name,date,manner_of_death,armed,age,gender,city,state,signs_of_mental_illness,...,share_hispanic,Race_Asian,Race_Black,Race_Hispanic,Race_Native_American,Race_Other,Race_White,threat_attack,threat_other,threat_undem
0,3,Tim Elliot,02/01/15,0,1,53.0,1,Shelton,WA,1,...,19.2,1,0,0,0,0,0,1,0,0
1,4,Lewis Lee Lembke,02/01/15,0,1,47.0,1,Aloha,OR,0,...,21.1,0,0,0,0,0,1,1,0,0
2,890,Phyllis Ilene Jepsen,02/10/15,0,1,55.0,0,Aloha,OR,1,...,21.1,0,0,0,0,0,1,0,1,0
3,5,John Paul Quintero,03/01/15,1,0,23.0,1,Wichita,KS,0,...,15.3,0,0,1,0,0,0,0,1,0
4,765,Nicholas Garner,22/08/15,0,1,26.0,1,Wichita,KS,0,...,15.3,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,2809,Sergio Velasco-Martinez,27/07/17,0,1,25.0,1,Avenal,CA,0,...,71.8,0,0,0,0,0,0,1,0,0
2199,2811,Matthew Folden,27/07/17,0,1,31.0,1,Wenatchee,WA,0,...,29.4,0,0,0,0,0,0,0,1,0
2200,2812,Alejandro Alvarado,27/07/17,0,1,NaN,1,Chowchilla,CA,0,...,37.8,0,0,1,0,0,0,1,0,0
2201,2818,Dennis W. Robinson,29/07/17,0,1,48.0,1,Melba,ID,0,...,24.6,0,0,0,0,0,0,1,0,0


In [47]:
df.to_csv("death_case2.csv")